id            为心跳信号分配的唯一标识
heartbeat_signals  心跳信号序列
label          心跳类别0、1、2、3

In [10]:
import pandas as pd
import numpy as np

In [11]:
train = pd.read_csv('./train.csv') 
testA = pd.read_csv('./testA.csv')
testB = pd.read_csv('./testB.csv')

In [12]:
train.tail()

,id,heartbeat_signals,label
99995,99995,"1.0,0.677705342021188,0.22239242747868546,0.25...",0.0
99996,99996,"0.9268571578157265,0.9063471198026871,0.636993...",2.0
99997,99997,"0.9258351628306013,0.5873839035878395,0.633226...",3.0
99998,99998,"1.0,0.9947621698382489,0.8297017704865509,0.45...",2.0
99999,99999,"0.9259994004527861,0.916476635326053,0.4042900...",0.0


In [13]:
testA.head()

,id,heartbeat_signals
0,100000,"0.9915713654170097,1.0,0.6318163407681274,0.13..."
1,100001,"0.6075533139615096,0.5417083883163654,0.340694..."
2,100002,"0.9752726292239277,0.6710965234906665,0.686758..."
3,100003,"0.9956348033996116,0.9170249621481004,0.521096..."
4,100004,"1.0,0.8879490481178918,0.745564725322326,0.531..."


In [14]:
testB.head()

,id,heartbeat_signals
0,120000,"1.0,0.9774807571353566,0.8272707700162286,0.58..."
1,120001,"0.4626720698152755,0.0,0.22602916098772136,0.3..."
2,120002,"0.6592562684599051,0.9778383713740022,0.959860..."
3,120003,"1.0,0.863413944079791,0.22081827785879307,0.17..."
4,120004,"0.992513249901794,0.8621977716587462,0.2911349..."


In [15]:
for col in train.columns:
    print(col)
    col_type=train[col].dtype
    print(col_type)
    if col_type !=object:
        c_min = train[col].min()
        c_max = train[col].max()
        print(c_min)
        print(c_max)

id
int64
0
99999
heartbeat_signals
object
label
float64
0.0
3.0


In [16]:
np.iinfo(np.int8)

iinfo(min=-128, max=127, dtype=int8)

In [17]:
#数据预处理   
#调整数据类型，解决占用内存过大的问题
def reduce_men_usage(df):
    start_mem=df.memory_usage().sum()/1204**2
    print("Memory usage of Dataframe is {:.2f} MB".format(start_mem))
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max=df[col].max()
            if str(col_type)[:3]=='int':
                if c_min >np.iinfo(np.int8).min and c_max<np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >np.iinfo(np.int16).min and c_max<np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >np.iinfo(np.int32).min and c_max<np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >np.iinfo(np.int64).min and c_max<np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min >np.finfo(np.float16).min and c_max<np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >np.finfo(np.float32).min and c_max<np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64) 
        else:
            df[col] =df[col].astype('category')
            
    end_mem = df.memory_usage().sum()/1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100*(start_mem-end_mem)/start_mem))
    
    return df

In [18]:
#预处理
train_list=[]
for items in train.values:
    train_list.append([items[0]]+[float(i) for i in items[1].split(',') +[items[2]]])

train = pd.DataFrame(np.array(train_list))
train.columns = ['id'] +['s'+str(i) for i in range(len(train_list[0])-2)] +['label']
train = reduce_men_usage(train)

train

Memory usage of Dataframe is 114.24 MB
Memory usage after optimization is: 39.67 MB
Decreased by 65.3%


,id,s0,s1,s2,s3,s4,s5,s6,s7,s8,...,s196,s197,s198,s199,s200,s201,s202,s203,s204,label
0,0.0,0.991211,0.943359,0.764648,0.618652,0.379639,0.190796,0.040222,0.026001,0.031708,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.971680,0.929199,0.572754,0.178467,0.122986,0.132324,0.094421,0.089600,0.030487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,1.000000,0.958984,0.701172,0.231812,0.000000,0.080688,0.128418,0.187500,0.280762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3.0,0.975586,0.934082,0.659668,0.249878,0.237061,0.281494,0.249878,0.249878,0.241455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.000000,0.055817,0.261230,0.359863,0.433105,0.453613,0.499023,0.542969,0.616699,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995.0,1.000000,0.677734,0.222412,0.257080,0.204712,0.054657,0.026154,0.118164,0.244873,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,99996.0,0.926758,0.906250,0.637207,0.415039,0.374756,0.382568,0.358887,0.341309,0.336426,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
99997,99997.0,0.925781,0.587402,0.633301,0.632324,0.639160,0.614258,0.599121,0.517578,0.403809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
99998,99998.0,1.000000,0.994629,0.829590,0.458252,0.264160,0.240234,0.213745,0.189331,0.203857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [19]:
testA_list=[]
for items in testA.values:
    testA_list.append([items[0]]+[float(i) for i in items[1].split(',')])
    
testA=pd.DataFrame(np.array(testA_list))
testA.columns=['id'] + ['s_'+str(i) for i in range(len(testA_list[0])-1)]

testA=reduce_men_usage(testA)
testA

Memory usage of Dataframe is 22.74 MB
Memory usage after optimization is: 7.90 MB
Decreased by 65.3%


,id,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,...,s_195,s_196,s_197,s_198,s_199,s_200,s_201,s_202,s_203,s_204
0,100000.0,0.991699,1.000000,0.631836,0.136230,0.041412,0.102722,0.120850,0.123413,0.107910,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,100001.0,0.607422,0.541504,0.340576,0.000000,0.090698,0.164917,0.195068,0.168823,0.198853,...,0.389893,0.386963,0.367188,0.364014,0.360596,0.357178,0.350586,0.350586,0.350586,0.36377
2,100002.0,0.975098,0.670898,0.686523,0.708496,0.718750,0.716797,0.720703,0.701660,0.596680,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,100003.0,0.995605,0.916992,0.520996,0.000000,0.221802,0.404053,0.490479,0.527344,0.518066,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,100004.0,1.000000,0.888184,0.745605,0.531738,0.380371,0.224609,0.091125,0.057648,0.003914,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,119995.0,1.000000,0.833008,0.634277,0.639160,0.624023,0.598145,0.613770,0.624023,0.628906,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
19996,119996.0,1.000000,0.826172,0.452148,0.082214,0.000000,0.137085,0.201050,0.165649,0.158081,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
19997,119997.0,0.951660,0.916504,0.667480,0.352051,0.255371,0.197388,0.173584,0.141968,0.134521,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
19998,119998.0,0.927734,0.677246,0.242920,0.055359,0.102112,0.072266,0.021011,0.038300,0.048553,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [20]:
testB_list=[]
for items in testB.values:
    testB_list.append([items[0]]+[float(i) for i in items[1].split(',')])
    
testB=pd.DataFrame(np.array(testB_list))
testB.columns=['id'] + ['s_'+str(i) for i in range(len(testA_list[0])-1)]

testB=reduce_men_usage(testB)
testB

Memory usage of Dataframe is 22.74 MB
Memory usage after optimization is: 7.90 MB
Decreased by 65.3%


,id,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,...,s_195,s_196,s_197,s_198,s_199,s_200,s_201,s_202,s_203,s_204
0,120000.0,1.000000,0.977539,0.827148,0.587402,0.336426,0.134033,0.098328,0.116089,0.075500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,120001.0,0.462646,0.000000,0.226074,0.398438,0.473633,0.480957,0.491943,0.517090,0.520508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,120002.0,0.659180,0.978027,0.959961,0.068726,0.068726,0.475342,0.481689,0.531250,0.506836,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,120003.0,1.000000,0.863281,0.220825,0.172729,0.395752,0.428711,0.450439,0.450439,0.447754,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,120004.0,0.992676,0.862305,0.291016,0.020752,0.218506,0.166260,0.084106,0.089722,0.100952,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,139995.0,0.755859,0.678711,0.544434,0.286133,0.100830,0.003160,0.000000,0.046692,0.094910,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,139996.0,1.000000,0.674316,0.716797,0.696777,0.696289,0.684570,0.619629,0.495850,0.346680,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,139997.0,0.727539,0.583008,0.545410,0.536133,0.512207,0.492676,0.445312,0.407959,0.345459,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,139998.0,0.623535,0.363525,0.328857,0.239502,0.120972,0.096252,0.104492,0.147827,0.137939,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
train.to_csv('train_pro.csv')
testA.to_csv('testA_pro.csv')
testB.to_csv('testB_pro.csv')